# importing nltk library

In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

# importing other libraries

In [2]:
!pip install PyPDF2
import PyPDF2
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.tokenize import sent_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


# load the training dataset

In [4]:
df=pd.read_csv('train.csv')
df.head()

,ABSTRACT,Label_val_2
0,a ever-growing datasets inside observational a...,Cosmology
1,we propose the framework considering optimal $...,Data Structures and Algorithms
2,nanostructures with open shell transition meta...,Strongly Correlated Electrons
3,stars are self-gravitating fluids inside which...,Fluid Dynamics
4,deep neural perception and control networks ar...,Computer vision and Pattern Recognition


# Total words before preprocessing(Feature-Extraction)

In [5]:
print("Total words before pre-processing :",df['ABSTRACT'].apply(lambda x: len(x.split(' '))).sum())

Total words before pre-processing : 2204798


# Pre-processing

In [7]:
stemmer = PorterStemmer()
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    return " ".join(filtered_words)
text=''
df['ABSTRACT']=df['ABSTRACT'].apply(preprocess)

In [8]:
df.head()

,ABSTRACT,Label_val_2
0,ever growing datasets inside observational ast...,Cosmology
1,propose framework considering optimal matching...,Data Structures and Algorithms
2,nanostructures open shell transition metal mol...,Strongly Correlated Electrons
3,stars self gravitating fluids inside pressure ...,Fluid Dynamics
4,deep neural perception control networks likely...,Computer vision and Pattern Recognition


# Training the model(Feature-selection)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

Total words after preprocessing

In [10]:
print("Total words after pre-processing :",df['ABSTRACT'].apply(lambda x: len(x.split(' '))).sum())

Total words after pre-processing : 1416411


# Oversampling and Train and test split

In [11]:
X=df.ABSTRACT
y=df.Label_val_2
X=X.values.reshape(1,-1)
X = X.transpose()

In [12]:
X.shape

(14004, 1)

In [13]:
! pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [14]:
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=0)
x_resampled,y_resampled=ros.fit_resample(X,y)

In [15]:
x_resampled=x_resampled.reshape(x_resampled.shape[:-1])
X_train,X_test,y_train,y_test=train_test_split(x_resampled,y_resampled, test_size=0.2,random_state=42)

In [16]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((47980,), (11995,), (47980,), (11995,))

# Training the model and classification

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

lr=Pipeline([('vect', CountVectorizer()),
             ('tfidf', TfidfTransformer()),
             ('clf', RandomForestClassifier())])
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
accuracy=accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test,y_pred))

Accuracy: 97.34%
                                              precision    recall  f1-score   support

                            Analysis of PDES       0.99      0.99      0.99       472
                                Applications       0.98      0.98      0.98       504
                     Artificial Intelligence       0.85      0.74      0.79       447
                    Astrophysics of Galaxies       1.00      0.99      1.00       457
                    Computation and Language       0.96      1.00      0.98       511
     Computer vision and Pattern Recognition       0.90      0.97      0.93       454
                                   Cosmology       1.00      1.00      1.00       466
              Data Structures and Algorithms       0.98      0.99      0.98       508
                       Differential Geometry       1.00      1.00      1.00       453
            Earth and Planetary Astrophysics       1.00      1.00      1.00       495
                              Fluid 

# Evaluating the model in the Test dataset

In [ ]:
X_test[:5]

In [19]:
y_test[:5]

47416      Representation Theory
6502     Artificial Intelligence
212      Artificial Intelligence
24330                  Cosmology
40360                Methodology
Name: Label_val_2, dtype: object

In [20]:
y_pred[:5]

array(['Representation Theory', 'Machine Learning',
       'Data Structures and Algorithms', 'Cosmology', 'Methodology'],
      dtype=object)

# Working of the Model(input)

In [33]:
pdf_dir = os.getcwd()
corpus=[]
file_name=[]
count=0
for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
      file_name.append([filename])
      count=count+1
      pdf_file = open(os.path.join(pdf_dir, filename), 'rb')
      pdf_reader = PyPDF2.PdfReader(pdf_file)
      pdftext = ''
      for i in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[i]
        pdftext+=page.extract_text()
      corpus.append(pdftext)
print("Number of Documents :",len(corpus))

Number of Documents : 7


# pre-processing

In [34]:
def listToString(s):
    str1 = " "
    # return string
    return (str1.join(s))

In [35]:
import numpy as np
preprocess_text=[preprocess(i) for i in corpus]
#print(len(preprocess_text))
def pos(s):
  c_text1=[]
  s=[s]
  for item in s:
    c_text1.extend(item.split())
  word=nltk.pos_tag(c_text1)
  noun_words=list()
  length=len(word)-1
  for i in range(0, length):
    if(word [i][1]=='NN' or word[i][1]=='NNS'or word[i][1]=='NNP'):
      noun_words.append(word[i][0])
  s = noun_words
  noun_words_text=listToString(s)
  return noun_words_text

# Predicted Domains of the Documents (Output)

In [36]:
print("Number of Documents :",len(corpus))
print("\n")
preprocess_text1=[pos(j) for j in preprocess_text]
#print(len(preprocess_text1))
pred=[]
print("Names of the documents :")
for n in file_name:
  print(n)
print("\n")
print("Predicted Domains :")
for i in preprocess_text1:
  i=[i]
  pred.append(lr.predict(i))
pred=np.array(pred)
pred1=pred.tolist()
for i in pred1:
  print(i)

Number of Documents : 7


Names of the documents :
['information-11-00075-v2.pdf']
['conference.pdf']
['More_2015_ApJ_806_2.pdf']
['155834422410170605651a-2495-2499-ShahidKhan-IT--FINAL-PAID_2.pdf']
['Muzzin_2012_ApJ_746_188.pdf']
['Social_Networks_and_Information_Systems_Ongoing_an.pdf']
['Most-cited20papers20paper.pdf']


Predicted Domains :
['Robotics']
['Artificial Intelligence']
['Astrophysics of Galaxies']
['Computation and Language']
['Astrophysics of Galaxies']
['Social and Information Networks']
['Machine Learning']


# New Features Implementation

In [37]:
def list_change(lis):
    newlist = []
    for element in lis:
        if type(element) is list:
            for item in element:
                newlist.append(item)
        else:
            newlist.append(element)
    return newlist


pred1_list = list_change(pred1)
print(pred1_list)

['Robotics', 'Artificial Intelligence', 'Astrophysics of Galaxies', 'Computation and Language', 'Astrophysics of Galaxies', 'Social and Information Networks', 'Machine Learning']


In [38]:
same_domain = []
other_domain = []
domain_counts = {}

for j in pred1_list:
    if j not in domain_counts:
        domain_counts[j] = 1
    else:
        domain_counts[j] += 1

for domain, count in domain_counts.items():
    if count > 1:
        for _ in range(count):
            same_domain.append(domain)
    else:
        other_domain.append(domain)
print("Same Domain")
print(same_domain)
print("\n")
print("Other Domain")
print(other_domain)

Same Domain
['Astrophysics of Galaxies', 'Astrophysics of Galaxies']


Other Domain
['Robotics', 'Artificial Intelligence', 'Computation and Language', 'Social and Information Networks', 'Machine Learning']


In [39]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
def generate_summary(text, n):
  sentences = sent_tokenize(text)
  stop_words = set(stopwords.words('english'))
  words = [word.lower() for word in word_tokenize(text) if word.lower() not in stop_words and word.isalnum()]
  word_freq = Counter(words)
  sentence_scores = {}

  for sentence in sentences:
    sentence_words = [word.lower() for word in word_tokenize(sentence) if word.lower() not in stop_words and word.isalnum()]
    sentence_score = sum([word_freq[word] for word in sentence_words])
    if len(sentence_words) < 30:
      sentence_scores[sentence] = sentence_score
      summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:n]
      summary = ' '.join(summary_sentences)

  return summary
count=1
for i in corpus:
  summary = generate_summary(i, 5)
  summary_sentences = summary.split('. ')
  formatted_summary = '.\n'.join(summary_sentences)
  print("Summary of Research Paper :",count)
  print(formatted_summary)
  print("\n")
  count=count+1

Summary of Research Paper : 1
Evaluation of social robot Matilta in an older adults care facility
shows that the companion robot has had a positive impact on older adults’ emotional well beings [ 6].
In addition,
we suggest conducting a long-term evaluation of social robots among older adults with two purposes:
understanding the effects of repeated interactions for older adults; ﬁnding and evaluating approaches
that aim to reduce repeated interactions with social robots.
Information gathered from external
environment sensors and older adults’ inputs can thus be used to help robots better understand
their surroundings and older adults’ demands.
The search strings were selected to capture all research pertaining to assistive or home robots for
elder care and smart home technologies for elder care.
Suggestions for
accelerating the deployment of robots at home for providing older adults with independent care in
the home are presented based on the results and insights from this study.


Sum

In [28]:
same_domain = []
same_domain_files = []
other_domain = []

domain_counts = {}
domain_files = {}

for j, file_names in (pred1_list, file_name_list):
    if j not in domain_counts:
        domain_counts[j] = 1
        domain_files[j] = [file_names]
    else:
        domain_counts[j] += 1
        domain_files[j].append(file_names)

for domain, count in domain_counts.items():
    if count > 1:
        for file_names in domain_files[domain]:
            same_domain_files.append(file_names)
    else:
        other_domain.append((domain, domain_files[domain][0]))

print(same_domain_files)
print(other_domain)

NameError: ignored

In [ ]:
def list_change(lis):
    flatList = []
    # Iterate with outer list
    for element in lis:
        if type(element) is list:
            # Check if type is list than iterate through the sublist
            for item in element:
                flatList.append(item)
        else:
            flatList.append(element)
    return flatList


pred1_list = pred1_
file_name_list=file_name
list_change(pred1_)
list_change(file_name)

# Visualization of the tf-idf

In [ ]:
c_text=list(map(preprocess,corpus))
print(c_text)

['see discussions ats author ofiles public ation researchgate public ation social networks information systems ongoing futu research streams article journal association inf ormation syst ems februar doi jais sour dblp citations reads author harri oinas kukk onen univ ersity oulu publica tions citations see profile kalle yytinen case west ern eser univ ersity publica tions citations see profile young jin case west ern eser univ ersity publica tions citations see profile ontent ollo wing age uplo aded kalle yytinen june user equest enhanc ement wnlo aded file volume issue article journal association information special issue abstract harri oinas kukkonen university oulu harri oinas kukkonen oulu kalle lyytinen case western reserve university kalle case edu youngjin yoo temple university yxyyoo gmail com review reasons increased interest network analyses organization studies information research also note impact new information technology capabilities increase terms improvements analysis 

In [ ]:
c_text1=[]
for item in c_text:
  c_text1.extend(item.split())
print(c_text1)

['see', 'discussions', 'ats', 'author', 'ofiles', 'public', 'ation', 'researchgate', 'public', 'ation', 'social', 'networks', 'information', 'systems', 'ongoing', 'futu', 'research', 'streams', 'article', 'journal', 'association', 'inf', 'ormation', 'syst', 'ems', 'februar', 'doi', 'jais', 'sour', 'dblp', 'citations', 'reads', 'author', 'harri', 'oinas', 'kukk', 'onen', 'univ', 'ersity', 'oulu', 'publica', 'tions', 'citations', 'see', 'profile', 'kalle', 'yytinen', 'case', 'west', 'ern', 'eser', 'univ', 'ersity', 'publica', 'tions', 'citations', 'see', 'profile', 'young', 'jin', 'case', 'west', 'ern', 'eser', 'univ', 'ersity', 'publica', 'tions', 'citations', 'see', 'profile', 'ontent', 'ollo', 'wing', 'age', 'uplo', 'aded', 'kalle', 'yytinen', 'june', 'user', 'equest', 'enhanc', 'ement', 'wnlo', 'aded', 'file', 'volume', 'issue', 'article', 'journal', 'association', 'information', 'special', 'issue', 'abstract', 'harri', 'oinas', 'kukkonen', 'university', 'oulu', 'harri', 'oinas', 'ku

In [ ]:
word=nltk.pos_tag(c_text1)
print(word)

[('see', 'VB'), ('discussions', 'NNS'), ('ats', 'VB'), ('author', 'NN'), ('ofiles', 'NNS'), ('public', 'JJ'), ('ation', 'NN'), ('researchgate', 'VB'), ('public', 'JJ'), ('ation', 'NN'), ('social', 'JJ'), ('networks', 'JJ'), ('information', 'NN'), ('systems', 'NNS'), ('ongoing', 'VBG'), ('futu', 'JJ'), ('research', 'NN'), ('streams', 'NNS'), ('article', 'NN'), ('journal', 'NN'), ('association', 'NN'), ('inf', 'JJ'), ('ormation', 'NN'), ('syst', 'NN'), ('ems', 'VBZ'), ('februar', 'JJ'), ('doi', 'NN'), ('jais', 'NN'), ('sour', 'JJ'), ('dblp', 'NN'), ('citations', 'NNS'), ('reads', 'VBP'), ('author', 'NN'), ('harri', 'NN'), ('oinas', 'IN'), ('kukk', 'NN'), ('onen', 'NN'), ('univ', 'JJ'), ('ersity', 'NN'), ('oulu', 'NN'), ('publica', 'NN'), ('tions', 'NNS'), ('citations', 'NNS'), ('see', 'VBP'), ('profile', 'JJ'), ('kalle', 'FW'), ('yytinen', 'NN'), ('case', 'NN'), ('west', 'JJS'), ('ern', 'JJ'), ('eser', 'NN'), ('univ', 'JJ'), ('ersity', 'NN'), ('publica', 'NN'), ('tions', 'NNS'), ('citati

In [ ]:
noun_words=list()
length=len(word)-1
for i in range(0, length):
  if(word [i][1]=='NN' or word[i][1]=='NNS'or word[i][1]=='NNP'):
    noun_words.append(word[i][0])
print(noun_words)

['discussions', 'author', 'ofiles', 'ation', 'ation', 'information', 'systems', 'research', 'streams', 'article', 'journal', 'association', 'ormation', 'syst', 'doi', 'jais', 'dblp', 'citations', 'author', 'harri', 'kukk', 'onen', 'ersity', 'oulu', 'publica', 'tions', 'citations', 'yytinen', 'case', 'eser', 'ersity', 'publica', 'tions', 'citations', 'jin', 'case', 'eser', 'ersity', 'publica', 'tions', 'citations', 'ollo', 'age', 'uplo', 'yytinen', 'june', 'user', 'enhanc', 'ement', 'wnlo', 'file', 'volume', 'issue', 'article', 'journal', 'association', 'information', 'issue', 'abstract', 'kukkonen', 'university', 'oulu', 'harri', 'kukkonen', 'oulu', 'kalle', 'lyytinen', 'case', 'reserve', 'university', 'kalle', 'case', 'edu', 'youngjin', 'yoo', 'temple', 'university', 'yxyyoo', 'gmail', 'com', 'review', 'reasons', 'interest', 'network', 'organization', 'studies', 'information', 'research', 'information', 'technology', 'capabilities', 'terms', 'improvements', 'analysis', 'ways', 'mainta

In [ ]:
df=pd.DataFrame(noun_words, columns=['noun_words'])

In [ ]:
v=TfidfVectorizer()
transformed_output=v.fit_transform(noun_words)
print(v.vocabulary_)

{'clustering': 482, 'boss': 305, 'galaxies': 1072, 'constraints': 579, 'miyatake': 1747, 'rachel': 2197, 'mandelbaum': 1610, 'masahiro': 1644, 'takada': 2715, 'david': 687, 'spergel': 2582, 'joel': 1424, 'brownstein': 333, 'kavli': 1447, 'physics': 2039, 'mathematics': 1659, 'wpi': 3045, 'todias': 2829, 'university': 2911, 'tokyo': 2834, 'chiba': 440, 'japandepartment': 1414, 'sciences': 2417, 'peyton': 2025, 'hall': 1174, 'princeton': 2123, 'usamcwilliams': 2916, 'cosmology': 634, 'department': 729, 'mellon': 1699, 'city': 460, 'astronomy': 177, 'astrophysics': 178, 'state': 2609, 'park': 1978, 'gravitation': 1144, 'cosmos': 636, 'pennsylvania': 2006, 'march': 1630, 'perform': 2014, 'joint': 1428, 'analysis': 105, 'abundance': 12, 'galaxy': 1076, 'data': 677, 'release': 2268, 'sloan': 2545, 'sky': 2542, 'survey': 2693, 'iii': 1290, 'baryon': 236, 'oscillation': 1933, 'spectroscopic': 2576, 'companion': 527, 'paper': 1969, 'signal': 2512, 'shape': 2483, 'catalog': 387, 'background': 21

In [ ]:
v=TfidfVectorizer()
tokenized_word=v.fit_transform(corpus)
feature_s=v.get_feature_names_out()
for i in feature_s:
  indx=v.vocabulary_.get(i)
  print(f"{i} {v.idf_[indx]}")

Streaming output truncated to the last 5000 lines.
claim 1.916290731874155
claimed 1.916290731874155
clarity 1.5108256237659907
clas 1.916290731874155
class 1.5108256237659907
classi 1.916290731874155
classical 1.916290731874155
classificatio 1.916290731874155
classification 1.916290731874155
classified 1.916290731874155
classifier 1.916290731874155
classifiers 1.916290731874155
classify 1.2231435513142097
classifying 1.5108256237659907
classiﬁcation 1.5108256237659907
classiﬁed 1.5108256237659907
classiﬁer 1.916290731874155
clean 1.916290731874155
cleaning 1.5108256237659907
clear 1.5108256237659907
clearly 1.916290731874155
cles 1.916290731874155
clf 1.916290731874155
climbing 1.916290731874155
clinicians 1.916290731874155
close 1.2231435513142097
closed 1.916290731874155
closely 1.5108256237659907
closer 1.916290731874155
clotet 1.916290731874155
cloud 1.5108256237659907
clue 1.916290731874155
clus 1.916290731874155
cluster 1.5108256237659907
clusterbecause 1.916290731874155
cluster

In [ ]:
df=pd.DataFrame(noun_words, columns=['Noun_words'])
pd.set_option("display.max_rows",None)
noun_frequencies=df['Noun_words'].value_counts()
noun_frequencies1=(noun_frequencies[:20])
noun_frequencies_dict=({'Noun_words':noun_frequencies1.index,'TF':noun_frequencies1.values})
noun_frequencies_df=pd.DataFrame(noun_frequencies_dict)
print(noun_frequencies_df)

     Noun_words   TF
0      galaxies  544
1          mass  331
2   environment  215
3        galaxy  125
4       cluster  119
5        figure   97
6         robot   94
7          star   92
8       density   91
9         model   89
10       sample   88
11         data   86
12     redshift   82
13   parameters   82
14     clusters   79
15     function   78
16     analysis   78
17        color   77
18     research   77
19       adults   77


In [ ]:
s=""
for i in noun_words:
  s+=i
print(s)

journaldoisocietyrightsgeminiclusterastrophysicssurveygclassroleenvironmentregulationgalaxyevolutionmuzzinwilsondavidgilbankhenkricardodemarcomichaelpietervandokkummarijnfranxericaellingsonamaliahicksallisonmarklacyalessandroretturajasonsuracetracydepartmentastronomyyaleuniversityadammuzzinyaleobservatoryuniversitydepartmentphysicsuniversitycaliforniadepartmentastronomyastrophysicsuniversitytorontoontariocanadaobservatoryobservatorysouthdepartmentastronomyconcepcioncasillaconcepcionchiledepartmentphysicsuniversitywaterloowaterloocanadacenterastrophysicsspaceastronomyuniversitycoloradoboulderdepartmentphysicsstateuniversitydepartmentphysicsmontrealcanadaalmasciencecenterheadquartersobservatoryspitzersciencecenterinstitutetechnologypasadenadecemberevaluateeffectsenvironmentmassgalaxypropertiessampleclusterﬁeldgalaxiesclusterastrophysicsspectroscopicsurveyconﬁrmgalaxiescorrelationsenvironmentpropertiesfractionfsfstarformationratesfrspeciﬁcsfrcolorplaceeffectsenvironmentmassgalaxiespropert

In [ ]:
def listToString(s):
    str1 = " "
    # return string
    return (str1.join(s))
s = noun_words
noun_words_text1=[listToString(s)]
print(noun_words_text1)

['journal doi society rights gemini cluster astrophysics survey gclass role environment regulation galaxy evolution muzzin wilson david gilbank henk ricardo demarco michael pieter van dokkum marijn franx erica ellingson amalia hicks allison mark lacy alessandro rettura jason surace tracy department astronomy yale university adam muzzin yale observatory university department physics university california department astronomy astrophysics university toronto ontario canada observatory observatory south department astronomy concepcion casilla concepcion chile department physics university waterloo waterloo canada center astrophysics space astronomy university colorado boulder department physics state university department physics montreal canada alma science center headquarters observatory spitzer science center institute technology pasadena december evaluate effects environment mass galaxy properties sample cluster ﬁeld galaxies cluster astrophysicsspectroscopic survey conﬁrm galaxies cor

In [ ]:
df.Label_val_2.value_counts()

Machine Learning                                2399
Artificial Intelligence                         1366
Computer vision and Pattern Recognition          789
Materials Science                                760
Strongly Correlated Electrons                    743
Analysis of PDES                                 607
Robotics                                         600
Applications                                     592
Astrophysics of Galaxies                         573
Earth and Planetary Astrophysics                 476
Data Structures and Algorithms                   462
Computation and Language                         442
Differential Geometry                            438
Cosmology                                        421
Number Theory Optimization                       386
Social and Information Networks                  382
Optimization and Control                         370
Information Theory                               318
Fluid Dynamics                                

In [ ]:
(df['Label_val_2'].value_counts()/14004)*100

Machine Learning                                17.130820
Artificial Intelligence                          9.754356
Computer vision and Pattern Recognition          5.634105
Materials Science                                5.427021
Strongly Correlated Electrons                    5.305627
Analysis of PDES                                 4.334476
Robotics                                         4.284490
Applications                                     4.227364
Astrophysics of Galaxies                         4.091688
Earth and Planetary Astrophysics                 3.399029
Data Structures and Algorithms                   3.299057
Computation and Language                         3.156241
Differential Geometry                            3.127678
Cosmology                                        3.006284
Number Theory Optimization                       2.756355
Social and Information Networks                  2.727792
Optimization and Control                         2.642102
Information Th

In [ ]:
listToString(noun_frequencies1)

TypeError: ignored

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
tokens = noun_words.split()
comment_words=''
for val in df.CONTENT:

    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                min_font_size = 10).generate(s)

# plot the WordCloud image
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

AttributeError: ignored